# ETL desde CRM: TRANSFORMACIÓN Y CARGA

## Conexión bbdds y configuración del proceso

In [1]:
# librerías a usar
import pandas as pd
import sqlite3
from datetime import datetime ,timedelta
from dateutil.relativedelta import relativedelta

In [2]:
# configuración pandas
pd.set_option('display.max_columns', None)

In [3]:
# apertura de la base de datos
conexion=sqlite3.connect("../data/final/dw_sf_tr.db")

In [4]:
# se genera cursor a la base de datos
cursor=conexion.cursor()

In [5]:
# fecha y hora de la carga
fec_modificacion=datetime.now()
fec_modificacion

datetime.datetime(2023, 6, 21, 12, 53, 26, 6089)

## Dimensión familia de producto

In [6]:
df_sf=pd.read_csv("../data/raw/DIM_CATEGORIA_PRODUCTO.CSV",sep=";")

In [7]:
df_sf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 2 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   IDE_CATEGORIA_PRODUCTO  2 non-null      object
 1   DES_CATEGORIA_PRODUCTO  2 non-null      object
dtypes: object(2)
memory usage: 160.0+ bytes


In [8]:
# se genera el dataframe de destino vacío
dim_FAMPRO=pd.read_sql_query("SELECT * FROM DIM_FAMPRO LIMIT 0", conexion)

In [9]:
# actualizamos los datos del dataframe
dim_FAMPRO["IDE_FAMPRO"]=df_sf["IDE_CATEGORIA_PRODUCTO"]
dim_FAMPRO["COD_FAMPRO"]=df_sf["IDE_CATEGORIA_PRODUCTO"]
dim_FAMPRO["DES_FAMPRO"]=df_sf["DES_CATEGORIA_PRODUCTO"]
dim_FAMPRO["FEC_MOD_FAMPRO"]=fec_modificacion
dim_FAMPRO.head(2)

,IDE_FAMPRO,COD_FAMPRO,DES_FAMPRO,FEC_MOD_FAMPRO
0,CAT001,CAT001,Belleza,2023-06-21 12:53:26.006089
1,CAT002,CAT002,Bienestar & Salud,2023-06-21 12:53:26.006089


In [10]:
# se genera la tabla de maniobra destino
dim_FAMPRO.to_sql('M_FAMPRO', conexion, index=False, if_exists='replace')

2

In [11]:
# se visualiza lo insertado
pd.read_sql_query("SELECT * FROM M_FAMPRO LIMIT 3", conexion)

,IDE_FAMPRO,COD_FAMPRO,DES_FAMPRO,FEC_MOD_FAMPRO
0,CAT001,CAT001,Belleza,2023-06-21 12:53:26.006089
1,CAT002,CAT002,Bienestar & Salud,2023-06-21 12:53:26.006089


In [12]:
# se actualiza o reemplaza la tabla destino
sql="""
    INSERT INTO DIM_FAMPRO
      SELECT * FROM M_FAMPRO WHERE True
      ON CONFLICT(IDE_FAMPRO) 
      DO UPDATE SET COD_FAMPRO=excluded.COD_FAMPRO,
                    DES_FAMPRO=excluded.DES_FAMPRO,
                    FEC_MOD_FAMPRO=excluded.FEC_MOD_FAMPRO;
    """
cursor.executescript(sql)

In [13]:
# se visualiza la tabla de destino
pd.read_sql_query("SELECT * FROM DIM_FAMPRO LIMIT 3", conexion)

,IDE_FAMPRO,COD_FAMPRO,DES_FAMPRO,FEC_MOD_FAMPRO
0,9999999999,9999999999,SIN FAMILIA,2001-01-01
1,CAT001,CAT001,Belleza,2023-06-21 12:53:26.006089
2,CAT002,CAT002,Bienestar & Salud,2023-06-21 12:53:26.006089


In [14]:
# commit
conexion.commit()

## Dimensión producto

In [15]:
# datos de origen
df_sf=pd.read_csv("../data/raw/DIM_PRODUCTO.CSV",sep=";")
df_sf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 3 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   IDE_PRODUCTO            7 non-null      object
 1   DES_PRODUCTO            7 non-null      object
 2   IDE_CATEGORIA_PRODUCTO  7 non-null      object
dtypes: object(3)
memory usage: 296.0+ bytes


In [16]:
# se genera el dataframe de destino vacío
dim_PRODUC=pd.read_sql_query("SELECT * FROM DIM_PRODUC LIMIT 0", conexion)

In [17]:
# se genera la tabla de posibles errores vacía
dim_PRODUC.to_sql('M_ERR_PRODUC', conexion, index=False, if_exists='replace')

0

In [18]:
# actualizamos los datos del dataframe
dim_PRODUC["IDE_PRODUC"]=df_sf["IDE_PRODUCTO"]
dim_PRODUC["COD_PRODUC"]=df_sf["IDE_PRODUCTO"]
dim_PRODUC["DES_PRODUC"]=df_sf["DES_PRODUCTO"]
dim_PRODUC["IDE_FAMPRO"]=df_sf["IDE_CATEGORIA_PRODUCTO"]
dim_PRODUC["FEC_MOD_PRODUC"]=fec_modificacion

In [19]:
# Cambiar los null de las llaves externas por el valor por defecto
dim_PRODUC["IDE_FAMPRO"]=dim_PRODUC["IDE_FAMPRO"].fillna(value="9999999999")

In [20]:
# se genera la tabla de maniobra destino
dim_PRODUC.to_sql('M_PRODUC', conexion, index=False, if_exists='replace')

7

In [21]:
# se visualiza lo insertado
pd.read_sql_query("SELECT * FROM M_PRODUC LIMIT 3", conexion)

,IDE_PRODUC,COD_PRODUC,DES_PRODUC,IDE_FAMPRO,FEC_MOD_PRODUC
0,PRO11,PRO11,11_FRAGANCIA_LUXURY,CAT001,2023-06-21 12:53:26.006089
1,PRO12,PRO12,12_DEPILADORA_ELECTRICA,CAT002,2023-06-21 12:53:26.006089
2,PRO13,PRO13,13_TENSIOMETRO_DIGITAL,CAT002,2023-06-21 12:53:26.006089


In [22]:
# se genera tabla con posibles errores por integridad referencia
sql="""
    INSERT INTO M_ERR_PRODUC
        SELECT P.* 
        FROM M_PRODUC P
        LEFT JOIN DIM_FAMPRO F USING (IDE_FAMPRO)
        WHERE F.IDE_FAMPRO IS NULL
         AND P.IDE_PRODUC NOT IN
             (SELECT IDE_PRODUC FROM M_ERR_PRODUC)
    """
cursor.executescript(sql)

In [23]:
# se visualiza los posibles errores
pd.read_sql_query("SELECT * FROM M_ERR_PRODUC LIMIT 3", conexion)

,IDE_PRODUC,COD_PRODUC,DES_PRODUC,IDE_FAMPRO,FEC_MOD_PRODUC


In [24]:
# se actualiza o reemplaza la tabla destino siempre que no esté el código de producto en la tabla de error
sql="""
    INSERT INTO DIM_PRODUC
      SELECT * FROM M_PRODUC 
      WHERE IDE_PRODUC NOT IN
          (SELECT IDE_PRODUC FROM M_ERR_PRODUC)
      ON CONFLICT(IDE_PRODUC) 
      DO UPDATE SET COD_PRODUC=excluded.COD_PRODUC,
                    DES_PRODUC=excluded.DES_PRODUC,
                    FEC_MOD_PRODUC=excluded.FEC_MOD_PRODUC;
    """
cursor.executescript(sql)

In [25]:
# se visualiza toda la tabla de destino
pd.read_sql_query("SELECT * FROM DIM_PRODUC LIMIT 3", conexion)

,IDE_PRODUC,COD_PRODUC,DES_PRODUC,IDE_FAMPRO,FEC_MOD_PRODUC
0,9999999999,9999999999,SIN PRODUCTO,9999999999,2001-01-01
1,PRO11,PRO11,11_FRAGANCIA_LUXURY,CAT001,2023-06-21 12:53:26.006089
2,PRO12,PRO12,12_DEPILADORA_ELECTRICA,CAT002,2023-06-21 12:53:26.006089


In [26]:
# commit
conexion.commit()

## Dimensión cuenta

In [27]:
# datos de origen
df_sf=pd.read_csv("../data/raw/DIM_CLIENTE_tr.CSV",sep=";")
df_sf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4638 entries, 0 to 4637
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   IDE_CLIENTE           4638 non-null   int64 
 1   IDE_CLIENTE_NIF       4638 non-null   object
 2   DES_CLIENTE           4638 non-null   object
 3   NUM_EDAD              4638 non-null   int64 
 4   NUM_TAMANO_FAMILIAR   4638 non-null   int64 
 5   NUM_ANOS_EXPERIENCIA  4638 non-null   int64 
 6   NUM_INGRESOS_ANO      4638 non-null   int64 
 7   IDE_NIVEL_EDUCACION   4638 non-null   int64 
dtypes: int64(6), object(2)
memory usage: 290.0+ KB


In [28]:
# se genera el dataframe de destino vacío
dim_cuenta=pd.read_sql_query("SELECT * FROM DIM_CUENTA LIMIT 0", conexion)

In [29]:
# actualizamos los datos del dataframe
dim_cuenta["IDE_CUENTA"]=df_sf["IDE_CLIENTE"]
dim_cuenta["COD_CUENTA"]=df_sf["IDE_CLIENTE"]
dim_cuenta["DES_CUENTA"]=df_sf["IDE_CLIENTE"]
dim_cuenta["DES_VIA_CUENTA"]="PASEO DE RECOLETOS, 1"
dim_cuenta["DES_CIUDAD_CUENTA"]="MADRID"
dim_cuenta["DES_ESTADO_CUENTA"]="COMUNIDAD DE MADRID"
dim_cuenta["DES_CODPOS_CUENTA"]="28007"
dim_cuenta["DES_PAIS_CUENTA"]="SPAIN"

dim_cuenta["NUM_INGRESOS_ANO_CUENTA"]=df_sf["NUM_INGRESOS_ANO"]
dim_cuenta["NUM_ANOS_EXPERIENCIA_CUENTA"]=df_sf["NUM_ANOS_EXPERIENCIA"]
dim_cuenta["NUM_TAMANO_FAMILIAR_CUENTA"]=df_sf["NUM_TAMANO_FAMILIAR"]
dim_cuenta["NUM_EDAD_CUENTA"]=df_sf["NUM_EDAD"]

dim_cuenta["FEC_MOD_CUENTA"]=fec_modificacion

In [30]:
# se genera la tabla de maniobra en destino
dim_cuenta.to_sql('M_CUENTA', conexion, index=False, if_exists='replace')

4638

In [31]:
# se visualiza lo insertado
pd.read_sql_query("SELECT * FROM M_CUENTA LIMIT 3", conexion)

,IDE_CUENTA,COD_CUENTA,DES_CUENTA,DES_VIA_CUENTA,DES_CIUDAD_CUENTA,DES_ESTADO_CUENTA,DES_CODPOS_CUENTA,DES_PAIS_CUENTA,NUM_EDAD_CUENTA,NUM_TAMANO_FAMILIAR_CUENTA,NUM_ANOS_EXPERIENCIA_CUENTA,NUM_INGRESOS_ANO_CUENTA,FEC_MOD_CUENTA
0,410,410,410,"PASEO DE RECOLETOS, 1",MADRID,COMUNIDAD DE MADRID,28007,SPAIN,60,1,30,22000,2023-06-21 12:53:26.006089
1,5393,5393,5393,"PASEO DE RECOLETOS, 1",MADRID,COMUNIDAD DE MADRID,28007,SPAIN,65,3,40,49000,2023-06-21 12:53:26.006089
2,5392,5392,5392,"PASEO DE RECOLETOS, 1",MADRID,COMUNIDAD DE MADRID,28007,SPAIN,63,2,39,24000,2023-06-21 12:53:26.006089


In [32]:
# se actualiza o reemplaza la tabla destino siempre que no esté el código de producto en la tabla de error
sql="""
    INSERT INTO DIM_CUENTA
      SELECT * FROM M_CUENTA 
      WHERE True
      ON CONFLICT(IDE_CUENTA) 
      DO UPDATE SET COD_CUENTA=excluded.COD_CUENTA,
                    DES_CUENTA=excluded.DES_CUENTA,
                    DES_VIA_CUENTA=excluded.DES_VIA_CUENTA,
                    DES_CIUDAD_CUENTA=excluded.DES_CIUDAD_CUENTA,
                    DES_ESTADO_CUENTA=excluded.DES_ESTADO_CUENTA,
                    DES_CODPOS_CUENTA=excluded.DES_CODPOS_CUENTA,
                    DES_PAIS_CUENTA=excluded.DES_PAIS_CUENTA,
                    NUM_INGRESOS_ANO_CUENTA=excluded.NUM_INGRESOS_ANO_CUENTA,
                    NUM_ANOS_EXPERIENCIA_CUENTA=excluded.NUM_ANOS_EXPERIENCIA_CUENTA,
                    NUM_TAMANO_FAMILIAR_CUENTA=excluded.NUM_TAMANO_FAMILIAR_CUENTA,
                    NUM_EDAD_CUENTA=excluded.NUM_EDAD_CUENTA,
                    FEC_MOD_CUENTA=excluded.FEC_MOD_CUENTA;
    """
cursor.executescript(sql)

In [33]:
# se visualiza toda la tabla de destino
pd.read_sql_query("SELECT * FROM DIM_CUENTA LIMIT 3", conexion)

,IDE_CUENTA,COD_CUENTA,DES_CUENTA,DES_VIA_CUENTA,DES_CIUDAD_CUENTA,DES_ESTADO_CUENTA,DES_CODPOS_CUENTA,DES_PAIS_CUENTA,NUM_EDAD_CUENTA,NUM_TAMANO_FAMILIAR_CUENTA,NUM_ANOS_EXPERIENCIA_CUENTA,NUM_INGRESOS_ANO_CUENTA,FEC_MOD_CUENTA
0,9999999999,9999999999,SIN CUENTA,None,None,None,None,None,NaN,NaN,NaN,NaN,2001-01-01
1,410,410,410,"PASEO DE RECOLETOS, 1",MADRID,COMUNIDAD DE MADRID,28007,SPAIN,60.0,1.0,30.0,22000.0,2023-06-21 12:53:26.006089
2,5393,5393,5393,"PASEO DE RECOLETOS, 1",MADRID,COMUNIDAD DE MADRID,28007,SPAIN,65.0,3.0,40.0,49000.0,2023-06-21 12:53:26.006089


In [34]:
# commit
conexion.commit()

## Dimensión contacto

In [35]:
# datos de origen
df_sf=pd.read_csv("../data/raw/DIM_CLIENTE_tr.CSV",sep=";")
df_sf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4638 entries, 0 to 4637
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   IDE_CLIENTE           4638 non-null   int64 
 1   IDE_CLIENTE_NIF       4638 non-null   object
 2   DES_CLIENTE           4638 non-null   object
 3   NUM_EDAD              4638 non-null   int64 
 4   NUM_TAMANO_FAMILIAR   4638 non-null   int64 
 5   NUM_ANOS_EXPERIENCIA  4638 non-null   int64 
 6   NUM_INGRESOS_ANO      4638 non-null   int64 
 7   IDE_NIVEL_EDUCACION   4638 non-null   int64 
dtypes: int64(6), object(2)
memory usage: 290.0+ KB


In [36]:
# se genera el dataframe de destino vacío
dim_CONTAC=pd.read_sql_query("SELECT * FROM DIM_CONTAC LIMIT 0", conexion)

In [37]:
# se genera la tabla de posibles errores vacía
dim_CONTAC.to_sql('M_ERR_CONTAC', conexion, index=False, if_exists='replace')

0

In [38]:
fecha_resultado = fec_modificacion.date() - timedelta(days=25*365)
fecha_resultado

datetime.date(1998, 6, 27)

In [39]:
# actualizamos los datos del dataframe
dim_CONTAC["IDE_CONTAC"]=df_sf["IDE_CLIENTE"]
dim_CONTAC["COD_CONTAC"]=df_sf["IDE_CLIENTE_NIF"]
dim_CONTAC["DES_CONTAC"]=df_sf["DES_CLIENTE"]
dim_CONTAC["DES_VIA_CONTAC"]="PASEO DE RECOLETOS, 1"
dim_CONTAC["DES_CIUDAD_CONTAC"]="MADRID"
dim_CONTAC["DES_ESTADO_CONTAC"]="COMUNIDAD DE MADRID"
dim_CONTAC["DES_CODPOS_CONTAC"]="28004"
dim_CONTAC["DES_PAIS_CONTAC"]="SPAIN"
dim_CONTAC["DES_EMAIL_CONTAC"]=df_sf["IDE_CLIENTE_NIF"]+"@mybusiness.com"
dim_CONTAC["DES_TELEFONO_CONTAC"]="+34916777888"
dim_CONTAC["DES_MOVIL_CONTAC"]="+34666777888"
dim_CONTAC["IND_EMAIL_RECHAZA_CONTAC"]="0"
dim_CONTAC["FEC_NACIMIENTO_CONTAC"] = df_sf['NUM_EDAD'].apply(lambda x: fec_modificacion.date() - relativedelta(years=x))
dim_CONTAC["IDE_CUENTA"]=df_sf["IDE_CLIENTE"]

dim_CONTAC["IDE_NIVEDU_CONTAC"]=df_sf["IDE_NIVEL_EDUCACION"]

dim_CONTAC["FEC_MOD_CONTAC"]=fec_modificacion
dim_CONTAC.head(3)

,IDE_CONTAC,COD_CONTAC,DES_CONTAC,DES_VIA_CONTAC,DES_CIUDAD_CONTAC,DES_ESTADO_CONTAC,DES_CODPOS_CONTAC,DES_PAIS_CONTAC,DES_EMAIL_CONTAC,DES_TELEFONO_CONTAC,DES_MOVIL_CONTAC,IND_EMAIL_RECHAZA_CONTAC,FEC_NACIMIENTO_CONTAC,IDE_CUENTA,IDE_NIVEDU_CONTAC,FEC_MOD_CONTAC
0,410,MIG100016,"PRUEBAAPELLIDO MIG100016, PRUEBANOMBRE MIG100016","PASEO DE RECOLETOS, 1",MADRID,COMUNIDAD DE MADRID,28004,SPAIN,MIG100016@mybusiness.com,+34916777888,+34666777888,0,1963-06-21,410,3,2023-06-21 12:53:26.006089
1,5393,MIG104999,"PRUEBAAPELLIDO MIG104999, PRUEBANOMBRE MIG104999","PASEO DE RECOLETOS, 1",MADRID,COMUNIDAD DE MADRID,28004,SPAIN,MIG104999@mybusiness.com,+34916777888,+34666777888,0,1958-06-21,5393,2,2023-06-21 12:53:26.006089
2,5392,MIG104998,"PRUEBAAPELLIDO MIG104998, PRUEBANOMBRE MIG104998","PASEO DE RECOLETOS, 1",MADRID,COMUNIDAD DE MADRID,28004,SPAIN,MIG104998@mybusiness.com,+34916777888,+34666777888,0,1960-06-21,5392,3,2023-06-21 12:53:26.006089


In [40]:
# Cambiar los null de las llaves externas por el valor por defecto
dim_CONTAC["IDE_CUENTA"]=dim_CONTAC["IDE_CUENTA"].fillna(value="9999999999")
dim_CONTAC["IDE_NIVEDU_CONTAC"]=dim_CONTAC["IDE_NIVEDU_CONTAC"].fillna(value="9999999999")

In [41]:
# se genera la tabla de maniobra destino
dim_CONTAC.to_sql('M_CONTAC', conexion, index=False, if_exists='replace')

4638

In [42]:
# se visualiza lo insertado
pd.read_sql_query("SELECT * FROM M_CONTAC LIMIT 2", conexion)

,IDE_CONTAC,COD_CONTAC,DES_CONTAC,DES_VIA_CONTAC,DES_CIUDAD_CONTAC,DES_ESTADO_CONTAC,DES_CODPOS_CONTAC,DES_PAIS_CONTAC,DES_EMAIL_CONTAC,DES_TELEFONO_CONTAC,DES_MOVIL_CONTAC,IND_EMAIL_RECHAZA_CONTAC,FEC_NACIMIENTO_CONTAC,IDE_CUENTA,IDE_NIVEDU_CONTAC,FEC_MOD_CONTAC
0,410,MIG100016,"PRUEBAAPELLIDO MIG100016, PRUEBANOMBRE MIG100016","PASEO DE RECOLETOS, 1",MADRID,COMUNIDAD DE MADRID,28004,SPAIN,MIG100016@mybusiness.com,+34916777888,+34666777888,0,1963-06-21,410,3,2023-06-21 12:53:26.006089
1,5393,MIG104999,"PRUEBAAPELLIDO MIG104999, PRUEBANOMBRE MIG104999","PASEO DE RECOLETOS, 1",MADRID,COMUNIDAD DE MADRID,28004,SPAIN,MIG104999@mybusiness.com,+34916777888,+34666777888,0,1958-06-21,5393,2,2023-06-21 12:53:26.006089


In [43]:
# se genera tabla con posibles errores por integridad referencia
sql="""
    INSERT INTO M_ERR_CONTAC
        SELECT P.* 
        FROM M_CONTAC P
        LEFT JOIN DIM_CUENTA F USING (IDE_CUENTA)
        WHERE F.IDE_CUENTA IS NULL
         AND P.IDE_CONTAC NOT IN
             (SELECT IDE_CONTAC FROM M_ERR_CONTAC)
    """
cursor.executescript(sql)

In [44]:
# se genera tabla con posibles errores por integridad referencia
sql="""
    INSERT INTO M_ERR_CONTAC
        SELECT P.* 
        FROM M_CONTAC P
        LEFT JOIN DIM_NIVEDU_CONTAC F USING (IDE_NIVEDU_CONTAC)
        WHERE F.IDE_NIVEDU_CONTAC IS NULL
         AND P.IDE_CONTAC NOT IN
             (SELECT IDE_CONTAC FROM M_ERR_CONTAC)
    """
cursor.executescript(sql)

In [45]:
# se visualiza los posibles errores
pd.read_sql_query("SELECT * FROM M_ERR_CONTAC", conexion)

,IDE_CONTAC,COD_CONTAC,DES_CONTAC,DES_VIA_CONTAC,DES_CIUDAD_CONTAC,DES_ESTADO_CONTAC,DES_CODPOS_CONTAC,DES_PAIS_CONTAC,DES_EMAIL_CONTAC,DES_TELEFONO_CONTAC,DES_MOVIL_CONTAC,IND_EMAIL_RECHAZA_CONTAC,FEC_NACIMIENTO_CONTAC,IDE_CUENTA,IDE_NIVEDU_CONTAC,FEC_MOD_CONTAC


In [46]:
# se actualiza o reemplaza la tabla destino siempre que no esté el código de producto en la tabla de error
sql="""
    INSERT INTO DIM_CONTAC
      SELECT * FROM M_CONTAC 
      WHERE IDE_CONTAC NOT IN
          (SELECT IDE_CONTAC FROM M_ERR_CONTAC)
      ON CONFLICT(IDE_CONTAC) 
      DO UPDATE SET COD_CONTAC=excluded.COD_CONTAC,
                    DES_CONTAC=excluded.DES_CONTAC,
                    DES_VIA_CONTAC=excluded.DES_VIA_CONTAC,
                    DES_CIUDAD_CONTAC=excluded.DES_CIUDAD_CONTAC,
                    DES_ESTADO_CONTAC=excluded.DES_ESTADO_CONTAC,
                    DES_CODPOS_CONTAC=excluded.DES_CODPOS_CONTAC,
                    DES_PAIS_CONTAC=excluded.DES_PAIS_CONTAC,
                    DES_EMAIL_CONTAC=excluded.DES_EMAIL_CONTAC,
                    DES_TELEFONO_CONTAC=excluded.DES_TELEFONO_CONTAC,
                    DES_MOVIL_CONTAC=excluded.DES_MOVIL_CONTAC,
                    IND_EMAIL_RECHAZA_CONTAC=excluded.IND_EMAIL_RECHAZA_CONTAC,
                    FEC_NACIMIENTO_CONTAC=excluded.FEC_NACIMIENTO_CONTAC,
                    IDE_NIVEDU_CONTAC=excluded.IDE_NIVEDU_CONTAC,
                    IDE_CUENTA=excluded.IDE_CUENTA,
                    FEC_MOD_CONTAC=excluded.FEC_MOD_CONTAC;
    """
cursor.executescript(sql)

In [47]:
# se visualiza toda la tabla de destino
pd.read_sql_query("SELECT * FROM DIM_CONTAC LIMIT 3", conexion)

,IDE_CONTAC,COD_CONTAC,DES_CONTAC,DES_VIA_CONTAC,DES_CIUDAD_CONTAC,DES_ESTADO_CONTAC,DES_CODPOS_CONTAC,DES_PAIS_CONTAC,DES_EMAIL_CONTAC,DES_TELEFONO_CONTAC,DES_MOVIL_CONTAC,IND_EMAIL_RECHAZA_CONTAC,FEC_NACIMIENTO_CONTAC,IDE_CUENTA,IDE_NIVEDU_CONTAC,FEC_MOD_CONTAC
0,9999999999,9999999999,SIN CONTACTO,None,None,None,None,None,None,None,None,None,None,9999999999,9999999999,2001-01-01
1,410,MIG100016,"PRUEBAAPELLIDO MIG100016, PRUEBANOMBRE MIG100016","PASEO DE RECOLETOS, 1",MADRID,COMUNIDAD DE MADRID,28004,SPAIN,MIG100016@mybusiness.com,+34916777888,+34666777888,0,1963-06-21,410,3,2023-06-21 12:53:26.006089
2,5393,MIG104999,"PRUEBAAPELLIDO MIG104999, PRUEBANOMBRE MIG104999","PASEO DE RECOLETOS, 1",MADRID,COMUNIDAD DE MADRID,28004,SPAIN,MIG104999@mybusiness.com,+34916777888,+34666777888,0,1958-06-21,5393,2,2023-06-21 12:53:26.006089


In [48]:
# commit
conexion.commit()

## Dimensión fecha oportunidad

In [49]:
# datos de origen
df_sf=pd.read_csv("../data/raw/FAC_VENTA_tr.CSV",sep=";")
df_sf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11354 entries, 0 to 11353
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   IDE_VENTA        11354 non-null  int64 
 1   IDE_PEDIDO       11354 non-null  int64 
 2   IDE_CLIENTE      11354 non-null  int64 
 3   IDE_FECHA        11354 non-null  object
 4   IDE_PRODUCTO     11354 non-null  object
 5   IDE_VENDEDOR     11354 non-null  int64 
 6   NUM_UNIDADES     11354 non-null  int64 
 7   NUM_PRECIO_NETO  11354 non-null  int64 
dtypes: int64(6), object(2)
memory usage: 709.8+ KB


In [50]:
df_sf.head()

,IDE_VENTA,IDE_PEDIDO,IDE_CLIENTE,IDE_FECHA,IDE_PRODUCTO,IDE_VENDEDOR,NUM_UNIDADES,NUM_PRECIO_NETO
0,2022,14281,4276,2022-07-16,PRO12,5,1,120
1,1959,13724,3719,2022-07-16,PRO12,1,1,120
2,1902,13158,3153,2022-07-16,PRO12,1,1,120
3,1647,10745,740,2022-07-16,PRO12,1,1,120
4,8181,13375,3370,2022-07-16,PRO15,1,1,200


In [51]:
# se genera el dataframe de destino vacío
dim_FEC_OPORTU=pd.read_sql_query("SELECT * FROM DIM_FEC_OPORTU LIMIT 0", conexion)

In [52]:
# obtener listas con las fechas (para columna fechas)
fechas=[]
if df_sf.shape[0]>0: # hay fechas a cargar
    for elt in df_sf["IDE_FECHA"]:
        if not elt is None:
            fecha=datetime.strptime(elt[:10], '%Y-%m-%d').date()
            print(fecha)
            fechas.append(fecha)
fechas

2022-07-16
2022-07-16
2022-07-16
2022-07-16
2022-07-16
2022-07-16
2022-07-16
2022-07-16
2022-07-16
2022-07-16
2022-07-16
2022-07-16
2022-07-16
2022-07-16
2022-07-16
2022-07-16
2022-07-16
2022-07-16
2022-07-16
2022-07-16
2022-07-16
2022-07-16
2022-07-16
2022-07-16
2022-07-16
2022-07-16
2022-07-16
2022-07-16
2022-07-16
2022-07-16
2022-07-16
2022-07-16
2022-07-16
2022-07-16
2022-07-16
2022-07-16
2022-07-16
2022-07-16
2022-07-16
2022-07-16
2022-07-16
2022-07-16
2022-07-16
2022-07-16
2022-07-16
2022-07-16
2022-07-16
2022-07-16
2022-07-16
2022-07-16
2022-07-16
2022-07-16
2022-07-16
2022-07-16
2022-07-16
2022-07-16
2022-07-16
2022-07-16
2022-07-16
2022-07-16
2022-07-16
2022-07-16
2022-07-16
2022-07-16
2022-07-16
2022-07-16
2022-07-16
2022-07-16
2022-07-16
2022-07-16
2022-07-16
2022-07-16
2022-07-16
2022-07-16
2022-07-16
2022-07-16
2022-07-16
2022-07-16
2022-07-16
2022-07-17
2022-07-17
2022-07-17
2022-07-17
2022-07-17
2022-07-17
2022-07-17
2022-07-17
2022-07-17
2022-07-17
2022-07-17
2022-07-17

[datetime.date(2022, 7, 16),
 datetime.date(2022, 7, 16),
 datetime.date(2022, 7, 16),
 datetime.date(2022, 7, 16),
 datetime.date(2022, 7, 16),
 datetime.date(2022, 7, 16),
 datetime.date(2022, 7, 16),
 datetime.date(2022, 7, 16),
 datetime.date(2022, 7, 16),
 datetime.date(2022, 7, 16),
 datetime.date(2022, 7, 16),
 datetime.date(2022, 7, 16),
 datetime.date(2022, 7, 16),
 datetime.date(2022, 7, 16),
 datetime.date(2022, 7, 16),
 datetime.date(2022, 7, 16),
 datetime.date(2022, 7, 16),
 datetime.date(2022, 7, 16),
 datetime.date(2022, 7, 16),
 datetime.date(2022, 7, 16),
 datetime.date(2022, 7, 16),
 datetime.date(2022, 7, 16),
 datetime.date(2022, 7, 16),
 datetime.date(2022, 7, 16),
 datetime.date(2022, 7, 16),
 datetime.date(2022, 7, 16),
 datetime.date(2022, 7, 16),
 datetime.date(2022, 7, 16),
 datetime.date(2022, 7, 16),
 datetime.date(2022, 7, 16),
 datetime.date(2022, 7, 16),
 datetime.date(2022, 7, 16),
 datetime.date(2022, 7, 16),
 datetime.date(2022, 7, 16),
 datetime.date

In [53]:
# obtener conjunto con las distintas fechas (sin duplicados)
fechas_dist=set(fechas)
fechas_dist

{datetime.date(2022, 7, 16),
 datetime.date(2022, 7, 17),
 datetime.date(2022, 7, 18),
 datetime.date(2022, 7, 19),
 datetime.date(2022, 7, 20),
 datetime.date(2022, 7, 21),
 datetime.date(2022, 7, 22),
 datetime.date(2022, 7, 23),
 datetime.date(2022, 7, 24),
 datetime.date(2022, 7, 25),
 datetime.date(2022, 7, 26),
 datetime.date(2022, 7, 27),
 datetime.date(2022, 7, 28),
 datetime.date(2022, 7, 29),
 datetime.date(2022, 7, 30),
 datetime.date(2022, 7, 31),
 datetime.date(2022, 8, 1),
 datetime.date(2022, 8, 2),
 datetime.date(2022, 8, 3),
 datetime.date(2022, 8, 4),
 datetime.date(2022, 8, 5),
 datetime.date(2022, 8, 6),
 datetime.date(2022, 8, 7),
 datetime.date(2022, 8, 8),
 datetime.date(2022, 8, 9),
 datetime.date(2022, 8, 10),
 datetime.date(2022, 8, 11),
 datetime.date(2022, 8, 12),
 datetime.date(2022, 8, 13),
 datetime.date(2022, 8, 14),
 datetime.date(2022, 8, 15),
 datetime.date(2022, 8, 16),
 datetime.date(2022, 8, 17),
 datetime.date(2022, 8, 18),
 datetime.date(2022, 8,

In [54]:
# actualizamos los datos del dataframe
dim_FEC_OPORTU["IDE_FEC_OPORTU"]=list(fechas_dist)
dim_FEC_OPORTU["COD_FEC_OPORTU"]=list(fechas_dist)
dim_FEC_OPORTU["DES_FEC_OPORTU"]=list(fechas_dist)
dim_FEC_OPORTU["IND_CARGADA_FEC_OPORTU"]="0"
dim_FEC_OPORTU["FEC_MOD_FEC_OPORTU"]=fec_modificacion
dim_FEC_OPORTU.head(3)

,IDE_FEC_OPORTU,COD_FEC_OPORTU,DES_FEC_OPORTU,IND_CARGADA_FEC_OPORTU,FEC_MOD_FEC_OPORTU
0,2022-08-28,2022-08-28,2022-08-28,0,2023-06-21 12:53:26.006089
1,2022-08-02,2022-08-02,2022-08-02,0,2023-06-21 12:53:26.006089
2,2022-08-23,2022-08-23,2022-08-23,0,2023-06-21 12:53:26.006089


In [55]:
# se genera la tabla de maniobra destino
dim_FEC_OPORTU.to_sql('M_FEC_OPORTU', conexion, index=False, if_exists='replace')

94

In [56]:
# se actualiza o reemplaza la tabla destino siempre que no esté el código de producto en la tabla de error
sql="""
    INSERT INTO DIM_FEC_OPORTU
      SELECT * FROM M_FEC_OPORTU 
      WHERE True
      ON CONFLICT(IDE_FEC_OPORTU) 
      DO UPDATE SET COD_FEC_OPORTU=excluded.COD_FEC_OPORTU,
                    DES_FEC_OPORTU=excluded.DES_FEC_OPORTU,
                    IND_CARGADA_FEC_OPORTU=excluded.IND_CARGADA_FEC_OPORTU,
                    FEC_MOD_FEC_OPORTU=excluded.FEC_MOD_FEC_OPORTU;
    """
cursor.executescript(sql)

In [57]:
# commit
conexion.commit()

## Dimensión Oportunidad

In [58]:
# se genera el dataframe de destino vacío
dim_OPORTU=pd.read_sql_query("SELECT * FROM DIM_OPORTU LIMIT 0", conexion)

In [59]:
# se genera la tabla de posibles errores vacía
dim_OPORTU.to_sql('M_ERR_OPORTU', conexion, index=False, if_exists='replace')

0

In [60]:
df_sf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11354 entries, 0 to 11353
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   IDE_VENTA        11354 non-null  int64 
 1   IDE_PEDIDO       11354 non-null  int64 
 2   IDE_CLIENTE      11354 non-null  int64 
 3   IDE_FECHA        11354 non-null  object
 4   IDE_PRODUCTO     11354 non-null  object
 5   IDE_VENDEDOR     11354 non-null  int64 
 6   NUM_UNIDADES     11354 non-null  int64 
 7   NUM_PRECIO_NETO  11354 non-null  int64 
dtypes: int64(6), object(2)
memory usage: 709.8+ KB


In [61]:
# actualizamos los datos del dataframe
dim_OPORTU["IDE_OPORTU"]=df_sf["IDE_PEDIDO"]
dim_OPORTU["COD_OPORTU"]=df_sf["IDE_PEDIDO"]
dim_OPORTU["DES_OPORTU"]=df_sf["IDE_PEDIDO"]
dim_OPORTU["IDE_FEC_OPORTU"]=df_sf["IDE_FECHA"]
dim_OPORTU["IDE_CUENTA"]=df_sf["IDE_CLIENTE"]
dim_OPORTU["IDE_CONTAC"]=df_sf["IDE_CLIENTE"]
dim_OPORTU["FEC_MOD_OPORTU"]=fec_modificacion
dim_OPORTU.head()

,IDE_OPORTU,COD_OPORTU,DES_OPORTU,IDE_FEC_OPORTU,IDE_CUENTA,IDE_CONTAC,FEC_MOD_OPORTU
0,14281,14281,14281,2022-07-16,4276,4276,2023-06-21 12:53:26.006089
1,13724,13724,13724,2022-07-16,3719,3719,2023-06-21 12:53:26.006089
2,13158,13158,13158,2022-07-16,3153,3153,2023-06-21 12:53:26.006089
3,10745,10745,10745,2022-07-16,740,740,2023-06-21 12:53:26.006089
4,13375,13375,13375,2022-07-16,3370,3370,2023-06-21 12:53:26.006089


In [62]:
# Cambiar los null de las llaves externas por el valor por defecto
dim_OPORTU["IDE_CUENTA"]=dim_OPORTU["IDE_CUENTA"].fillna(value="9999999999")
dim_OPORTU["IDE_CONTAC"]=dim_OPORTU["IDE_CONTAC"].fillna(value="9999999999")
dim_OPORTU["IDE_FEC_OPORTU"]=dim_OPORTU["IDE_FEC_OPORTU"].fillna(value="2001-01-01")

In [63]:
dim_OPORTU=dim_OPORTU.drop_duplicates()
dim_OPORTU.count()

IDE_OPORTU        4638
COD_OPORTU        4638
DES_OPORTU        4638
IDE_FEC_OPORTU    4638
IDE_CUENTA        4638
IDE_CONTAC        4638
FEC_MOD_OPORTU    4638
dtype: int64

In [64]:
# se genera la tabla de maniobra destino
dim_OPORTU.to_sql('M_OPORTU', conexion, index=False, if_exists='replace')

4638

In [65]:
# se visualiza lo insertado
pd.read_sql_query("SELECT * FROM M_OPORTU LIMIT 3", conexion)

,IDE_OPORTU,COD_OPORTU,DES_OPORTU,IDE_FEC_OPORTU,IDE_CUENTA,IDE_CONTAC,FEC_MOD_OPORTU
0,14281,14281,14281,2022-07-16,4276,4276,2023-06-21 12:53:26.006089
1,13724,13724,13724,2022-07-16,3719,3719,2023-06-21 12:53:26.006089
2,13158,13158,13158,2022-07-16,3153,3153,2023-06-21 12:53:26.006089


In [66]:
# se genera tabla con posibles errores por integridad referencial cuentas
sql="""
    INSERT INTO M_ERR_OPORTU
        SELECT P.* 
        FROM M_OPORTU P
        LEFT JOIN DIM_CUENTA F USING (IDE_CUENTA)
        WHERE F.IDE_CUENTA IS NULL
         AND P.IDE_OPORTU NOT IN
             (SELECT IDE_OPORTU FROM M_ERR_OPORTU)
    """
cursor.executescript(sql)

In [67]:
# se genera tabla con posibles errores por integridad referencial contactos
sql="""
    INSERT INTO M_ERR_OPORTU
        SELECT P.* 
        FROM M_OPORTU P
        LEFT JOIN DIM_CONTAC F USING (IDE_CONTAC)
        WHERE F.IDE_CONTAC IS NULL
         AND P.IDE_OPORTU NOT IN
             (SELECT IDE_OPORTU FROM M_ERR_OPORTU)
    """
cursor.executescript(sql)

In [68]:
# se genera tabla con posibles errores por integridad referencial fecha oportunidad
sql="""
    INSERT INTO M_ERR_OPORTU
        SELECT P.* 
        FROM M_OPORTU P
        LEFT JOIN DIM_FEC_OPORTU F USING (IDE_FEC_OPORTU)
        WHERE F.IDE_FEC_OPORTU IS NULL
         AND P.IDE_OPORTU NOT IN
             (SELECT IDE_OPORTU FROM M_ERR_OPORTU)
    """
cursor.executescript(sql)

In [69]:
# se visualiza los posibles errores
pd.read_sql_query("SELECT * FROM M_ERR_OPORTU", conexion)

,IDE_OPORTU,COD_OPORTU,DES_OPORTU,IDE_FEC_OPORTU,IDE_CUENTA,IDE_CONTAC,FEC_MOD_OPORTU


In [70]:
# se actualiza o reemplaza la tabla destino siempre que no esté el código de producto en la tabla de error
sql="""
    INSERT INTO DIM_OPORTU
      SELECT * FROM M_OPORTU
      WHERE IDE_OPORTU NOT IN
          (SELECT IDE_OPORTU FROM M_ERR_OPORTU)
      ON CONFLICT(IDE_OPORTU) 
      DO UPDATE SET COD_OPORTU=excluded.COD_OPORTU,
                    DES_OPORTU=excluded.DES_OPORTU,
                    IDE_FEC_OPORTU=excluded.IDE_FEC_OPORTU,
                    IDE_CUENTA=excluded.IDE_CUENTA,                    
                    IDE_CONTAC=excluded.IDE_CONTAC,                                        
                    FEC_MOD_OPORTU=excluded.FEC_MOD_OPORTU;
    """
cursor.executescript(sql)

In [71]:
# commit
conexion.commit()

## Hecho Línea Oportunidad

In [72]:
df_sf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11354 entries, 0 to 11353
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   IDE_VENTA        11354 non-null  int64 
 1   IDE_PEDIDO       11354 non-null  int64 
 2   IDE_CLIENTE      11354 non-null  int64 
 3   IDE_FECHA        11354 non-null  object
 4   IDE_PRODUCTO     11354 non-null  object
 5   IDE_VENDEDOR     11354 non-null  int64 
 6   NUM_UNIDADES     11354 non-null  int64 
 7   NUM_PRECIO_NETO  11354 non-null  int64 
dtypes: int64(6), object(2)
memory usage: 709.8+ KB


In [73]:
# se genera el dataframe de destino vacío
fac_LIN_OPORTU=pd.read_sql_query("SELECT * FROM FAC_LIN_OPORTU LIMIT 0", conexion)

In [74]:
# se genera la tabla de posibles errores vacía
fac_LIN_OPORTU.to_sql('M_ERR_LIN_OPORTU', conexion, index=False, if_exists='replace')

0

In [75]:
# actualizamos los datos del dataframe
fac_LIN_OPORTU["IDE_LIN_OPORTU"]=df_sf["IDE_VENTA"]
fac_LIN_OPORTU["COD_LIN_OPORTU"]=df_sf["IDE_VENTA"]
fac_LIN_OPORTU["DES_LIN_OPORTU"]=df_sf["IDE_VENTA"]
fac_LIN_OPORTU["NUM_CANTIDAD_LIN_OPORTU"]=df_sf["NUM_UNIDADES"]
fac_LIN_OPORTU["NUM_PRECTOT_LIN_OPORTU"]=df_sf["NUM_PRECIO_NETO"]
fac_LIN_OPORTU["NUM_PRECUNI_LIN_OPORTU"]=round(df_sf["NUM_PRECIO_NETO"]/df_sf["NUM_UNIDADES"],0)
fac_LIN_OPORTU["IDE_PRODUC"]=df_sf["IDE_PRODUCTO"]
fac_LIN_OPORTU["IDE_OPORTU"]=df_sf["IDE_PEDIDO"]
fac_LIN_OPORTU["FEC_MOD_LIN_OPORTU"]=fec_modificacion

In [76]:
# Cambiar los null de las llaves externas por el valor por defecto
fac_LIN_OPORTU["IDE_PRODUC"]=fac_LIN_OPORTU["IDE_PRODUC"].fillna(value="9999999999")
fac_LIN_OPORTU["IDE_OPORTU"]=fac_LIN_OPORTU["IDE_OPORTU"].fillna(value="9999999999")

In [77]:
# se genera la tabla de maniobra destino
fac_LIN_OPORTU.to_sql('M_LIN_OPORTU', conexion, index=False, if_exists='replace')

11354

In [78]:
# se visualiza lo insertado
pd.read_sql_query("SELECT * FROM M_LIN_OPORTU LIMIT 3", conexion)

,IDE_LIN_OPORTU,COD_LIN_OPORTU,DES_LIN_OPORTU,NUM_CANTIDAD_LIN_OPORTU,NUM_PRECTOT_LIN_OPORTU,NUM_PRECUNI_LIN_OPORTU,IDE_PRODUC,IDE_OPORTU,FEC_MOD_LIN_OPORTU
0,2022,2022,2022,1,120,120.0,PRO12,14281,2023-06-21 12:53:26.006089
1,1959,1959,1959,1,120,120.0,PRO12,13724,2023-06-21 12:53:26.006089
2,1902,1902,1902,1,120,120.0,PRO12,13158,2023-06-21 12:53:26.006089


In [79]:
# se genera tabla con posibles errores por integridad referencial oportunidad
sql="""
    INSERT INTO M_ERR_LIN_OPORTU
        SELECT P.* 
        FROM M_LIN_OPORTU P
        LEFT JOIN DIM_OPORTU F USING (IDE_OPORTU)
        WHERE F.IDE_OPORTU IS NULL
         AND P.IDE_LIN_OPORTU NOT IN
             (SELECT IDE_LIN_OPORTU FROM M_ERR_LIN_OPORTU)
    """
cursor.executescript(sql)

In [80]:
# se genera tabla con posibles errores por integridad referencial producto
sql="""
    INSERT INTO M_ERR_LIN_OPORTU
        SELECT P.* 
        FROM M_LIN_OPORTU P
        LEFT JOIN DIM_PRODUC F USING (IDE_PRODUC)
        WHERE F.IDE_PRODUC IS NULL
         AND P.IDE_LIN_OPORTU NOT IN
             (SELECT IDE_LIN_OPORTU FROM M_ERR_LIN_OPORTU)
    """
cursor.executescript(sql)

In [81]:
# se visualiza los posibles errores
pd.read_sql_query("SELECT * FROM M_ERR_LIN_OPORTU", conexion)

,IDE_LIN_OPORTU,COD_LIN_OPORTU,DES_LIN_OPORTU,NUM_CANTIDAD_LIN_OPORTU,NUM_PRECTOT_LIN_OPORTU,NUM_PRECUNI_LIN_OPORTU,IDE_PRODUC,IDE_OPORTU,FEC_MOD_LIN_OPORTU


In [82]:
# se actualiza o reemplaza la tabla destino siempre que no esté el código de producto en la tabla de error
sql="""
    INSERT INTO FAC_LIN_OPORTU
      SELECT * FROM M_LIN_OPORTU
      WHERE IDE_LIN_OPORTU NOT IN
          (SELECT IDE_LIN_OPORTU FROM M_ERR_OPORTU)
      ON CONFLICT(IDE_LIN_OPORTU) 
      DO UPDATE SET COD_LIN_OPORTU=excluded.COD_LIN_OPORTU,
                    DES_LIN_OPORTU=excluded.DES_LIN_OPORTU,
                    NUM_CANTIDAD_LIN_OPORTU=excluded.NUM_CANTIDAD_LIN_OPORTU,
                    NUM_PRECTOT_LIN_OPORTU=excluded.NUM_PRECTOT_LIN_OPORTU,
                    NUM_PRECUNI_LIN_OPORTU=excluded.NUM_PRECUNI_LIN_OPORTU,
                    IDE_PRODUC=excluded.IDE_PRODUC,                    
                    IDE_OPORTU=excluded.IDE_OPORTU,                                        
                    FEC_MOD_LIN_OPORTU=excluded.FEC_MOD_LIN_OPORTU;
    """
cursor.executescript(sql)

In [83]:
# commit
conexion.commit()

## Actualización final dimensión Fecha Oportunidad

In [84]:
# Se indica que la carga ha finalizado
sql="""
    UPDATE DIM_FEC_OPORTU
      SET IND_CARGADA_FEC_OPORTU='1'
      WHERE IDE_FEC_OPORTU IN 
            (SELECT IDE_FEC_OPORTU FROM M_FEC_OPORTU)
;
    """
cursor.executescript(sql)

In [85]:
# commit
conexion.commit()